    You have a maximum budget of 200.
    A roster consists of exactly 15 players.
    A roster has 1 QB slot, 3 WR slots, 2 RB slots, 1 TE slot, 1 K slot, 1 DEF slot, and 6 Bench slots.
    Any QB, WR, RB, TE, K, DEF slot can only contain a player of that type.
    A Bench slot can contain any player type.
    A player on your roster can only be assigned to one slot.
    You want at most 2 QB and 2 TE on your roster.
    You want at most 1 K and 1 DEF on your roster.
    A player on your bench will only generate a fraction of their projected points. A bench player should generate ½d of their projected points where d is their depth at their position. For example, if you have RB1, RB2, RB3, and RB4 on your team with projected points P1, P2, P3, and P4 the expected points from these players would be $P1 + P2 + ½ * P2 + ¼ * P3$
    

In [4]:
!pip install ortools

     |████████████████████████████████| 14.5 MB 4.8 MB/s 
     |████████████████████████████████| 126 kB 57.5 MB/s 
     |████████████████████████████████| 1.1 MB 43.2 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.12.0
    Uninstalling absl-py-0.12.0:
      Successfully uninstalled absl-py-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.4.0 requires absl-py<0.13,>=0.9, but you have absl-py 1.0.0 which is incompatible.


In [13]:
import pandas as pd
from ortools.linear_solver import pywraplp

#tried other solvers but didn't work
solver = pywraplp.Solver.CreateSolver('SCIP')
df = pd.read_csv("fantasy_football_data.csv")


act_roster = {
    'QB': 1,
    'WR': 3,
    'RB': 2,
    'TE': 1,
    'K': 1,
    'DEF': 1
}

depth_chart = {
    'QB': 2,
    'WR': 7,
    'RB': 7,
    'TE': 2,
    'K': 1,
    'DEF': 1
}





In [14]:
def objective(index):
    name = df.Name[index]
    pos = df.Position[index]
    values = []
    for depth in range(depth_chart[pos]):
        # if player is a starter, use the normal points
        if act_roster[pos] <= depth:
            values.append(sub_dir[pos][(name, depth)] * list(df.ProjectedPoints)[index])
        # if player is a bench player, use the discounted list(df.ProjectedPoints)
        else:
            values.append(sub_dir[pos][(name, depth)] * list(df.ProjectedPoints)[index] * (1/pow(2, depth-act_roster[pos])))
    return sum(values)

In [15]:
def solve_prob():

  status = solver.Solve()

  if status == solver.OPTIMAL:
      starters = []
      bench = []
      for pos in set(list(df.Position)):
          for key in sub_dir[pos].keys():
              if sub_dir[pos][key].solution_value() == 1:
                  if key[1] < act_roster[pos]:
                      starters.append(key)
                  else:
                      bench.append(key)
  return starters, bench

In [34]:
def print_results():
  starters,bench = solve_prob()
  print("My team: \n")
  print("Starters:")
  for starter in starters:
      print(f"{starter[0]} - {df.Position[df.Name == starter[0]].iloc[0]}")
  print()

  print("Bench:")
  for warmer in bench:
      print(f"{warmer[0]} - {df.Position[df.Name == warmer[0]].iloc[0]}")
  print()

  starter_points = sum([df.ProjectedPoints[df.Name == starter[0]].iloc[0] for starter in starters])
  bench_points = 0
  for warmer in bench:
      pos = df.Position[df.Name == warmer[0]].iloc[0]
      bench_points += df.ProjectedPoints[df.Name == warmer[0]].iloc[0] * (1 / pow(2, warmer[1] - act_roster[pos]))
  print(f'Points obtained: {starter_points + bench_points:.0f}')
  print()
  print(f"budget used: {sum([df.Price[df.Name == player[0]].iloc[0] for player in (starters + bench)]):.2f}")


In [21]:
#All the constraints 
# dictionary acting kind of as a tree, easier to check each player in each position and reduces the if statements using it this way

sub_dir = {}
for pos in set(list(df.Position)):
  #check all players by position
  sub_dir[pos] = ({(df.Name[i],depth): solver.IntVar(0, 1, df.iloc[i, 0] + "_" + str(depth)) for i in df.index[df.Position == pos] for depth in range(depth_chart[pos])})
  #the objective function where we want to maximize or projected points
solver.Maximize(sum([objective(i) for i in df.index]))
solver.Add(sum([sub_dir[pos][(df.Name[i], depth)] for pos in set(list(df.Position)) for i in df.index[df.Position == pos] for depth in range(depth_chart[pos]) ]) == 15)

# stay under budget
solver.Add(sum([sub_dir[pos][(player, depth)] * df.Price[df.Name == player].iloc[0] for pos in set(list(df.Position)) for player in set(df.Name[df.Position == pos]) for depth in range(depth_chart[pos])]) <= 200)

for pos in set(list(df.Position)):
    solver.Add(sum([sub_dir[pos][(player, 0)] for player in set(df.Name[df.Position == pos])]) >= 1) # must have starter 
    solver.Add(sum([sub_dir[pos][(player, 0)] for player in set(df.Name[df.Position == pos])]) <= act_roster[pos]) # limit starters 
    solver.Add(sum([sub_dir[pos][k] for k in sub_dir[pos].keys()]) <= depth_chart[pos]) # max per pos

    # 1 player 
    for depth in range(depth_chart[pos]):
        solver.Add(sum([sub_dir[pos][(player, depth)] for player in set(df.Name[df.Position == pos])]) <= 1)

for player in set(df.Name):
    pos = df.Position[df.Name == player].iloc[0]
    solver.Add(sum([sub_dir[pos][(player, depth)] for depth in range(depth_chart[pos])]) <= 1) # not same player




In [35]:
print_results()

My team: 

Starters:
Le'Veon Bell - RB
C.J. Anderson - RB
Tyrod Taylor - QB
Adam Vinatieri - K
Eric Ebron - TE
Antonio Brown - WR
T.Y. Hilton - WR
Tyrell Williams - WR
Houston Texans - DEF

Bench:
Jonathan Stewart - RB
Danny Woodhead - RB
Derrick Henry - RB
Theo Riddick - RB
Duke Johnson - RB
Carson Palmer - QB

Points obtained: 1867

budget used: 200.00
